# [EX4] 멋진 작사가 만들기

#### * 목표
작사하는 모델 만들기

#### * 목차
1. 데이터 다운로드      
2. 라이브러리 import    
3. 데이터 준비       
 1) 데이터 읽어오기      
 2) 데이터 정제     
 3) tokenizer 및 padding     
 4) tensor, tokenizer 확인      
4. 평가 데이터셋 분리       
 1) 소스문장, 타겟문장 생성   
 2) train, test test 분리    
5. 모델 학습 준비      
 1) Embedding 레이어 생성        
 2) 데이터셋 객체 생성    
 3) model에 데이터 배치 테스트     
 4) summary 확인        
6. 모델 학습     
7. 모델 테스트     
8. 회고        

## 1. 데이터 다운로드

In [1]:
# wget https://d3s0tskafalll9.cloudfront.net/media/documents/song_lyrics.zip
# unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics

# 또는 https://www.kaggle.com/paultimothymooney/poetry/data 에서 다운로드

print('download success')

download success


## 2. 라이브러리 Import

In [2]:
import glob
import os
import re
import tensorflow as tf   
from sklearn.model_selection import train_test_split

#import numpy as np         
#import tensorflow as tf    

print('import success')

import success


## 3. 데이터 준비

#### 1) 데이터 읽어오기

In [3]:
txt_file_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/*' # 파일 경로

txt_list = glob.glob(txt_file_path) # glob 모듈을 사용하여 파일 내용 읽기

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담기
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() # txt 파일 읽기
        raw_corpus.extend(raw) # raw_corpus 리스트에 문장 단위로 저장

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3]) # 문장 확인

데이터 크기: 187088
Examples:
 ['At first I was afraid', 'I was petrified', 'I kept thinking I could never live without you']


#### 2) 데이터 정제 

In [4]:
# 정규표현식(Regex)을 이용한 필터링
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자로 바꾸고, 양쪽 공백을 지움
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개의 공백은 하나의 공백으로 바꿈
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) #  a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿈
    sentence = sentence.strip() # 다시 양쪽 공백을 지움
    sentence = '<start> ' + sentence + ' <end>' # 문장 시작에는 <start>, 끝에는 <end>를 추가
    return sentence
    
# corpus에 정제 문장 모으기
corpus = []

# 원하지 않는 문장은 건너뛰기
for sentence in raw_corpus:
    if len(sentence) == 0: continue  # 길이가 0인 문장 제외
    if sentence[-1] == ":": continue # : 로 끝나는 문장 제외
    if len(sentence.split(' ')) > 13: continue # 13개 단어 이상 문장 제외
                
# 정제 후 담기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
corpus[:10] # 정제 결과 확인

['<start> at first i was afraid <end>',
 '<start> i was petrified <end>',
 '<start> i kept thinking i could never live without you <end>',
 '<start> by my side but then i spent so many nights <end>',
 '<start> just thinking how you ve done me wrong <end>',
 '<start> i grew strong <end>',
 '<start> i learned how to get along and so you re back <end>',
 '<start> from outer space <end>',
 '<start> i just walked in to find you <end>',
 '<start> i would have made you leave your key <end>']

#### 3) tokenizer 및 padding

In [5]:
# 텐서플로우의 Tokenizer와 pad_sequences를 사용
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 7000,  # 7000단어를 기억할 수 있는 tokenizer
        filters = ' ', # 텍스트에서 필터링될 문자인 문자열
        oov_token = "<unk>" # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿈
    )
    
    # tokenize() 함수로 데이터를 Tensor로 변환
    tokenizer.fit_on_texts(corpus) # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tensor = tokenizer.texts_to_sequences(corpus) # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춤
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   71  256 ...    0    0    0]
 [   2    5   57 ...    0    0    0]
 [   2    5 1135 ...    0    0    0]
 ...
 [   2    8    5 ...    0    0    0]
 [   2   46   16 ...    0    0    0]
 [   2    6  176 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f437ddd35d0>


#### 4) tensor, tokenizer 확인

In [6]:
# 생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력
# 2은 <start>, 3은 <end>, 0은 <pad>
print(tensor[:3, :10])
print()

# 단어사전 구축 확인(텐서 데이터는 모두 정수)
# tokenizer에 구축된 단어 사전의 인덱스
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

[[   2   71  256    5   57  666    3    0    0    0]
 [   2    5   57 6459    3    0    0    0    0    0]
 [   2    5 1135  512    5  104   79  200  257    7]]

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


## 4. 평가 데이터셋 분리

#### 1) 소스문장, 타겟 문장 생성

In [7]:
src_input = tensor[:, :-1] # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
tgt_input = tensor[:, 1:] # tensor에서 <start>를 잘라내서 타겟 문장을 생성   

print(src_input[0])
print(tgt_input[0])

[  2  71 256   5  57 666   3   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[ 71 256   5  57 666   3   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


#### 2) train, test set 분리

In [8]:
# sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리
enc_train, enc_val, dec_train, dec_val  = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          random_state=32)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape) # <end> 제외

Source Train: (130250, 32)
Target Train: (130250, 32)


## 5. 모델 학습 준비

#### 1) Embedding 레이어 생성

In [9]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        # embedding_size 는 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

embedding_size = 32 # 벡터 차원의 수, 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있음
hidden_size = 2048 # LSTM 레이어의 hidden state 의 차원수
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

print('Embedding success')

Embedding success


#### 2) 데이터셋 객체 생성

In [10]:
# tf.data.Dataset.from_tensor_slices() 메소드를 이용해 tf.data.Dataset객체를 생성
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듬
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 32), (256, 32)), types: (tf.int32, tf.int32)>

#### 3) model에 데이터 배치 테스트

In [11]:
# model의 input shape가 결정되면서 model.build()가 자동으로 호출
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣기
model(src_sample)

<tf.Tensor: shape=(256, 32, 7001), dtype=float32, numpy=
array([[[ 6.98107215e-06, -3.43116371e-05,  1.00308243e-05, ...,
          2.15480486e-05, -1.47061601e-05, -3.35423465e-06],
        [ 1.04622295e-05, -8.11904320e-05,  1.90454411e-05, ...,
         -2.62516533e-05, -9.38203157e-05, -6.80318844e-05],
        [-2.12608556e-05, -1.15545765e-04,  6.19280254e-05, ...,
         -9.55341384e-05, -1.62852099e-04, -1.82818578e-04],
        ...,
        [-4.99359041e-04,  6.41416991e-04, -1.69942575e-03, ...,
          7.79174035e-04,  7.41559139e-04,  1.24906213e-03],
        [-4.95993823e-04,  6.41874503e-04, -1.70822919e-03, ...,
          7.82408868e-04,  7.47190265e-04,  1.29128573e-03],
        [-4.93068306e-04,  6.41717692e-04, -1.71450654e-03, ...,
          7.84918549e-04,  7.51683256e-04,  1.32837961e-03]],

       [[ 6.98107215e-06, -3.43116371e-05,  1.00308243e-05, ...,
          2.15480486e-05, -1.47061601e-05, -3.35423465e-06],
        [ 1.04622295e-05, -8.11904320e-05,  1.

#### 4) summary 확인

In [12]:
model.summary()
# Output Shape : 모델은 입력 시퀀스의 길이를 모르기 때문에 Output Shape를 특정할 수 없다.

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  133019    
_________________________________________________________________
lstm (LSTM)                  multiple                  16941056  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  14345049  
Total params: 64,981,748
Trainable params: 64,981,748
Non-trainable params: 0
_________________________________________________________________


## 6. 모델 학습

In [13]:
# 사이킷런 학습
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

history = model.fit(
    enc_train,
    dec_train,
    batch_size = 256, # 배치 size
    epochs = 10, # 10번 학습
    validation_data=(enc_val, dec_val),
)

Epoch 1/10
509/509 [==============================] - 952s 2s/step - loss: 1.9905 - val_loss: 1.3669
Epoch 2/10
509/509 [==============================] - 949s 2s/step - loss: 1.4184 - val_loss: 1.3495
Epoch 3/10
509/509 [==============================] - 949s 2s/step - loss: 1.3168 - val_loss: 1.2937
Epoch 4/10
509/509 [==============================] - 948s 2s/step - loss: 1.2525 - val_loss: 1.2582
Epoch 5/10
509/509 [==============================] - 948s 2s/step - loss: 1.1967 - val_loss: 1.2253
Epoch 6/10
509/509 [==============================] - 948s 2s/step - loss: 1.1388 - val_loss: 1.1972
Epoch 7/10
509/509 [==============================] - 947s 2s/step - loss: 1.0787 - val_loss: 1.1735
Epoch 8/10
509/509 [==============================] - 946s 2s/step - loss: 1.0220 - val_loss: 1.1490
Epoch 9/10
509/509 [==============================] - 947s 2s/step - loss: 0.9573 - val_loss: 1.1296
Epoch 10/10
509/509 [==============================] - 945s 2s/step - loss: 0.8994 - val_lo

## 7. 모델 테스트

In [14]:
# generate_text 함수는 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor) # 입력받은 문장의 텐서를 입력
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]  # 예측된 값 중 가장 높은 확률인 word index를 뽑음
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis = 0)], axis =- 1) # 2에서 예측된 word index를 문장 뒤에 붙
        # 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마침
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '

## 8. 회고

#### 1. '토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외' 방법 고민
if len(sentence.split(' ')) > 13: continue       
 -> start와 end를 제외한, 단어 13개 이상을 가진 문장은 필터링 하였음 

#### 2. tensorflow의 model.fit() 코드 고민
 -> 아래 사이트에서 인자 참고하여 작성     
[참고] https://www.tensorflow.org/guide/keras/train_and_evaluate?hl=ko

#### 3. RNN - LSTM 이해
 -> embedding_size 와 hidden_size 의 값 지정의 여러번의 시도가 있었다     
 -> input, output 의 개념 더 자세히 알아봐야겠다     
[참고] https://engineer-mole.tistory.com/22